In [419]:
import agentpy as ap
import random as rdm

In [420]:
class CleaningRobot(ap.Agent):

    def setup(self):
        self.moves = 0
        self.nextMove = None

    def getNextMove(self, x, y, boundX, boundY, unavailable):
        options = self.possible_moves(
            x,
            y,
            boundX, 
            boundY,
            unavailable
        )
        
        if len(options) != 0:
            self.nextMove = self.model.random.choices(options)[0]

    def possible_moves(self, x, y, boundX, boundY, unavailable):
        a = []
        validMoves = []

        for possibleX in [x, x + 1, x - 1]:
            for possibleY in [y, y + 1, y - 1]:
                a.append((possibleX, possibleY))
                if ((possibleX, possibleY) not in unavailable) and\
                    (possibleX < boundX and possibleX >= 0) and\
                    (possibleY < boundY and possibleY >= 0) and\
                    (possibleY != y or possibleX != x):
                        validMoves.append((possibleX, possibleY))

        return validMoves


In [421]:
class CleaningModel(ap.Model):
    
    def setup(self):

#Create agents (robots)
        self.agents = ap.AgentList(
            self,
            self.p["agents"], 
            CleaningRobot
        )

#Create grid(donde limpian)
        self.floor = ap.Grid(
            self,
            (self.p['M'], self.p['N']), track_empty=True, check_border=False
        )

        self.floor.add_agents(
            self.agents, [(1,1)] * self.p["agents"], empty=True
        )

#Create dirty tiles
#Estado 0: Limpio, 1: Sucio
        self.dirtyTiles = 0
        self.tilesLeft = 0
        self.time = self.p["steps"]
        self.floor.add_field("state", -1)

        #Creates tiles in random pos
        for y in range(self.p['M']):
            for x in range(self.p['N']):
                self.floor["state"][y][x] = rdm.choices((0,1), [1-self.p["avgSucio"], self.p["avgSucio"]])[0]
                
                if self.floor["state"][y][x] == 1:
                    self.dirtyTiles += 1

        self.tilesLeft = self.dirtyTiles

    def step(self):
        for clean in self.agents:
            x = self.floor.positions[clean][0]
            y = self.floor.positions[clean][1]

            # tile is dirty.
            if self.floor["state"][y][x] == 1:
                self.floor["state"][y][x] = 0
                self.tilesLeft -= 1

            else: # tile is clean
                if clean.nextMove == (x, y) or clean.nextMove is None: # future move hasn't been calc.
                    clean.getNextMove(
                        x,
                        y,
                        self.p['M'],
                        self.p['N'],
                        [
                            self.floor.positions[neighbor]
                            for neighbor in self.floor.neighbors(clean)
                        ]
                    )

                else: # future move is already calc.
                    clean.moves += 1
                    self.floor.move_to(clean, clean.nextMove)

        self.time -= 1

        if self.tilesLeft == 0 or self.time < 1:
            self.stop()

    def end(self):
        self.report(
            ["steps"],
            self.p["steps"] - self.time
        )

        self.report(
            ["moves"],
            self.agents[0].moves
        )

        self.report(["cleaned"], str(int((self.dirtyTiles - self.tilesLeft) / self.dirtyTiles * 100)) + '%')



In [422]:
# Define parameters
parameters = {
    'agents': 10,
    'M': 15,
    'N': 15,
    'steps': 200,
    'avgSucio': .9
}


In [423]:
model = CleaningModel(parameters)
results = model.run()


Completed: 200 steps
Run time: 0:00:00.308838
Simulation finished


In [424]:
print(results.reporters["steps"][0])
print(results.reporters["cleaned"][0])
print(results.reporters["moves"][0])


200
81%
92
